# 🎫 Support Ticket Classifier

**Goal**: Classify customer support messages into Billing, Technical, or Account

**Method**: TF-IDF + Logistic Regression (traditional NLP)

**Improvements**:
- ✅ Refined weak supervision labeling (better Account/Technical distinction)
- ✅ Balanced class weights to handle imbalanced data
- ✅ Label leakage mitigation (exclude labeling keywords from features)
- ✅ Cross-validation for robust evaluation
- ✅ Simple preprocessing (no external dependencies)

In [22]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import joblib
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries loaded successfully!")

✓ Libraries loaded successfully!


## 1. Load Data

In [23]:
df = pd.read_csv('twcs.csv')
print(f"Total records: {len(df):,}")

# Filter inbound customer messages
df = df[df['inbound'] == True].copy()
print(f"Inbound customer messages: {len(df):,}")

Total records: 2,811,774
Inbound customer messages: 1,537,843


## 2. Improved Weak Supervision Labeling

**Strategy to improve Account precision**:
- **Stronger Account signals**: Focus on authentication/access issues (login, password, locked, verify)
- **Priority rules**: Technical errors override weak Account matches
- **Stricter matching**: Drop ambiguous cases (ties)

In [24]:
# Refined keyword sets with less overlap
LABELING_KEYWORDS = {
    'billing': [
        'bill', 'billing', 'charge', 'charged', 'payment', 'invoice', 'refund',
        'subscription', 'pay', 'paid', 'card', 'credit', 'debit', 'price', 'cost',
        'fee', 'receipt', 'transaction', 'overcharge', 'cancel subscription'
    ],
    
    'technical': [
        'error', 'crash', 'crashed', 'crashing', 'bug', 'issue', 'problem',
        'broken', 'not working', 'doesnt work', 'slow', 'loading', 'update',
        'ios', 'android', 'app', 'website', 'battery', 'freeze', 'frozen', 'lag',
        'download', 'install', 'version', 'device', 'glitch', 'wifi', 'connection'
    ],
    
    'account': [
        'login', 'log in', 'sign in', 'signin', 'password', 'reset password',
        'forgot password', 'username', 'account locked', 'locked out', 'verify',
        'verification', 'verification code', 'cant access', 'deactivate', 'reactivate',
        'email change', '2fa', 'two factor', 'authentication', 'security code'
    ]
}

# Strong indicators for priority-based labeling
STRONG_TECHNICAL = ['crash', 'error', 'bug', 'not working', 'broken', 'freeze']
STRONG_ACCOUNT = ['login', 'log in', 'password', 'locked', 'verification code', '2fa']

def assign_label(text):
    """Priority-based labeling with reduced Account/Technical overlap"""
    if pd.isna(text):
        return None
    
    text_lower = text.lower()
    
    # Check strong signals first
    has_strong_account = any(phrase in text_lower for phrase in STRONG_ACCOUNT)
    has_strong_technical = any(phrase in text_lower for phrase in STRONG_TECHNICAL)
    
    # If both strong signals present, drop (ambiguous)
    if has_strong_account and has_strong_technical:
        return None
    
    # Priority: strong signals win
    if has_strong_account:
        return 'Account'
    if has_strong_technical:
        return 'Technical'
    
    # Default: keyword counting
    billing_score = sum(1 for kw in LABELING_KEYWORDS['billing'] if kw in text_lower)
    technical_score = sum(1 for kw in LABELING_KEYWORDS['technical'] if kw in text_lower)
    account_score = sum(1 for kw in LABELING_KEYWORDS['account'] if kw in text_lower)
    
    scores = [billing_score, technical_score, account_score]
    max_score = max(scores)
    
    # No match or tie → drop
    if max_score == 0 or scores.count(max_score) > 1:
        return None
    
    if billing_score == max_score:
        return 'Billing'
    elif technical_score == max_score:
        return 'Technical'
    else:
        return 'Account'

# Apply labeling
df['label'] = df['text'].apply(assign_label)
df = df[df['label'].notna()].copy()

print(f"\n✓ Labeled records: {len(df):,}")
print(f"\nLabel distribution:")
print(df['label'].value_counts())
print(f"\nClass proportions:")
print((df['label'].value_counts() / len(df) * 100).round(2))


✓ Labeled records: 537,748

Label distribution:
label
Technical    357426
Billing      158454
Account       21868
Name: count, dtype: int64

Class proportions:
label
Technical    66.47
Billing      29.47
Account       4.07
Name: count, dtype: float64


## 3. Simple Text Cleaning

**No external dependencies** - using only basic Python and regex

In [25]:
# Simple English stopwords (most common)
STOPWORDS = set([
    'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for',
    'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are', 'were', 'be',
    'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will',
    'would', 'should', 'could', 'may', 'might', 'must', 'can', 'this',
    'that', 'these', 'those', 'i', 'you', 'he', 'she', 'it', 'we', 'they',
    'my', 'your', 'his', 'her', 'its', 'our', 'their', 'me', 'him', 'us',
    'them', 'what', 'which', 'who', 'when', 'where', 'why', 'how', 'all',
    'each', 'every', 'both', 'few', 'more', 'most', 'some', 'such', 'no',
    'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 'just'
])

def clean_text(text):
    """Simple text cleaning without external dependencies"""
    if pd.isna(text):
        return ""
    
    # Lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # Keep only letters, numbers, and spaces
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove stopwords and short words
    words = [w for w in text.split() if w not in STOPWORDS and len(w) > 2]
    
    return ' '.join(words)

df['text_clean'] = df['text'].apply(clean_text)
df = df[df['text_clean'].str.len() > 0].copy()
print(f"✓ Records after cleaning: {len(df):,}")
print(f"\nExample:")
print(df[['text', 'text_clean', 'label']].head(2))

✓ Records after cleaning: 534,088

Example:
                                                 text  \
12  @115714 y’all lie about your “great” connectio...   
23  actually that's a broken link you sent me and ...   

                                           text_clean      label  
12  lie about great connection bars lte still won ...  Technical  
23    actually broken link sent incorrect information  Technical  


## 4. Train/Validation/Test Split

In [26]:
# Sample 50k for training
SAMPLE_SIZE = min(50000, len(df))
df_sample = df.sample(n=SAMPLE_SIZE, random_state=42)

X = df_sample['text_clean']
y = df_sample['label']

# Split: 70% train, 15% validation, 15% test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)

print(f"Training:   {len(X_train):,} ({len(X_train)/len(X)*100:.1f}%)")
print(f"Validation: {len(X_val):,} ({len(X_val)/len(X)*100:.1f}%)")
print(f"Test:       {len(X_test):,} ({len(X_test)/len(X)*100:.1f}%)")
print(f"\nTraining set distribution:")
print(pd.Series(y_train).value_counts())

Training:   35,020 (70.0%)
Validation: 7,480 (15.0%)
Test:       7,500 (15.0%)

Training set distribution:
label
Technical    23283
Billing      10278
Account       1459
Name: count, dtype: int64


## 5. TF-IDF Vectorization with Label Leakage Mitigation

**Critical**: Exclude labeling keywords from features to prevent circular reasoning

In [27]:
# Collect all labeling keywords to exclude
all_labeling_keywords = set()
for keywords in LABELING_KEYWORDS.values():
    for kw in keywords:
        # Add both original and space-removed versions
        all_labeling_keywords.add(kw.replace(' ', ''))
        all_labeling_keywords.add(kw)

# Combine with stopwords
custom_stop_words = list(STOPWORDS) + list(all_labeling_keywords)

print(f"Excluding {len(all_labeling_keywords)} labeling keywords from TF-IDF features")

# TF-IDF with bigrams
vectorizer = TfidfVectorizer(
    max_features=6000,
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.7,
    stop_words=custom_stop_words
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

print(f"\n✓ TF-IDF shape: {X_train_tfidf.shape}")
print(f"✓ Features extracted: {len(vectorizer.get_feature_names_out())}")

Excluding 81 labeling keywords from TF-IDF features

✓ TF-IDF shape: (35020, 6000)
✓ Features extracted: 6000


## 6. Train Model with Balanced Class Weights

**Fix for class imbalance**: `class_weight='balanced'` automatically adjusts weights inversely proportional to class frequencies

In [28]:
# Light hyperparameter tuning
param_grid = {'C': [0.5, 1.0, 2.0, 4.0]}

model_base = LogisticRegression(
    max_iter=3000,
    solver='lbfgs',
    class_weight='balanced',  # CRITICAL for imbalanced data
    random_state=42
)

print("Running GridSearchCV with class_weight='balanced'...")
grid_search = GridSearchCV(
    model_base,
    param_grid,
    cv=3,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_tfidf, y_train)

print(f"\n✓ Best C parameter: {grid_search.best_params_['C']}")
print(f"✓ Best CV F1 Score: {grid_search.best_score_:.4f}")

model = grid_search.best_estimator_

Running GridSearchCV with class_weight='balanced'...
Fitting 3 folds for each of 4 candidates, totalling 12 fits

✓ Best C parameter: 2.0
✓ Best CV F1 Score: 0.7389


## 7. Cross-Validation

In [29]:
print("\n5-Fold Cross-Validation...")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X_train_tfidf, y_train, cv=cv, scoring='f1_macro')

print(f"CV F1-Scores: {cv_scores}")
print(f"Mean CV F1: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")


5-Fold Cross-Validation...
CV F1-Scores: [0.73687004 0.74001974 0.73464274 0.73883905 0.74456985]
Mean CV F1: 0.7390 (+/- 0.0067)


## 8. Validation Set Performance

In [30]:
y_val_pred = model.predict(X_val_tfidf)

print("="*60)
print("VALIDATION SET PERFORMANCE")
print("="*60)
print(f"Accuracy:  {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Macro F1:  {f1_score(y_val, y_val_pred, average='macro'):.4f}\n")
print(classification_report(y_val, y_val_pred))

VALIDATION SET PERFORMANCE
Accuracy:  0.8221
Macro F1:  0.7544

              precision    recall  f1-score   support

     Account       0.55      0.77      0.64       312
     Billing       0.71      0.80      0.75      2195
   Technical       0.91      0.83      0.87      4973

    accuracy                           0.82      7480
   macro avg       0.72      0.80      0.75      7480
weighted avg       0.84      0.82      0.83      7480



## 9. Test Set Evaluation (FINAL METRICS)

**Required metrics**: Accuracy, Macro Precision, Macro Recall, Macro F1, Confusion Matrix, Classification Report

In [31]:
y_test_pred = model.predict(X_test_tfidf)

label_order = ['Billing', 'Technical', 'Account']

# Calculate metrics
accuracy = accuracy_score(y_test, y_test_pred)
macro_precision = precision_score(y_test, y_test_pred, average='macro', labels=label_order)
macro_recall = recall_score(y_test, y_test_pred, average='macro', labels=label_order)
macro_f1 = f1_score(y_test, y_test_pred, average='macro', labels=label_order)

print("="*60)
print("TEST SET PERFORMANCE (FINAL METRICS)")
print("="*60)
print(f"Accuracy:           {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Macro Precision:    {macro_precision:.4f} ({macro_precision*100:.2f}%)")
print(f"Macro Recall:       {macro_recall:.4f} ({macro_recall*100:.2f}%)")
print(f"Macro F1-Score:     {macro_f1:.4f} ({macro_f1*100:.2f}%)")

print("\n" + "="*60)
print("CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_test, y_test_pred, labels=label_order))

print("="*60)
print("CONFUSION MATRIX")
print("="*60)
cm = confusion_matrix(y_test, y_test_pred, labels=label_order)
print(cm)
print(f"\nLabel order: {label_order}")

# Highlight Account precision
from sklearn.metrics import precision_recall_fscore_support
prec, rec, f1, sup = precision_recall_fscore_support(y_test, y_test_pred, labels=label_order)
account_idx = label_order.index('Account')
print(f"\n🎯 ACCOUNT PRECISION: {prec[account_idx]:.4f} ({prec[account_idx]*100:.1f}%)")
print(f"   Target: > 0.55 (55%)")

TEST SET PERFORMANCE (FINAL METRICS)
Accuracy:           0.8049 (80.49%)
Macro Precision:    0.6966 (69.66%)
Macro Recall:       0.7703 (77.03%)
Macro F1-Score:     0.7261 (72.61%)

CLASSIFICATION REPORT
              precision    recall  f1-score   support

     Billing       0.69      0.78      0.73      2201
   Technical       0.90      0.82      0.86      4986
     Account       0.50      0.71      0.59       313

    accuracy                           0.80      7500
   macro avg       0.70      0.77      0.73      7500
weighted avg       0.82      0.80      0.81      7500

CONFUSION MATRIX
[[1727  398   76]
 [ 755 4089  142]
 [  30   62  221]]

Label order: ['Billing', 'Technical', 'Account']

🎯 ACCOUNT PRECISION: 0.5034 (50.3%)
   Target: > 0.55 (55%)


## 10. Export Model

In [32]:
# Create pipeline
pipeline = Pipeline([
    ('tfidf', vectorizer),
    ('clf', model)
])

# Save
joblib.dump(pipeline, 'model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
print("✓ Model saved as model.pkl")
print("✓ Vectorizer saved as tfidf_vectorizer.pkl")

✓ Model saved as model.pkl
✓ Vectorizer saved as tfidf_vectorizer.pkl


## 11. Demo Predictions on sample.csv

In [33]:
# Load sample data
df_demo = pd.read_csv('sample.csv')
df_demo['text_clean'] = df_demo['text'].apply(clean_text)
df_demo = df_demo[df_demo['text_clean'].str.len() > 0]

# Predict first 10 rows
demo_texts = df_demo['text'].head(10).values
demo_clean = df_demo['text_clean'].head(10).values
demo_tfidf = vectorizer.transform(demo_clean)
predictions = model.predict(demo_tfidf)

print("="*80)
print("DEMO PREDICTIONS (sample.csv - First 10 rows)")
print("="*80)
for i, (text, pred) in enumerate(zip(demo_texts, predictions), 1):
    print(f"\n[{i}] {text[:70]}...")
    print(f"    → Predicted: {pred}")
print("\n" + "="*80)
print("✓ Demo complete!")

DEMO PREDICTIONS (sample.csv - First 10 rows)

[1] @AppleSupport causing the reply to be disregarded and the tapped notif...
    → Predicted: Technical

[2] @105835 Your business means a lot to us. Please DM your name, zip code...
    → Predicted: Billing

[3] @76328 I really hope you all change but I'm sure you won't! Because yo...
    → Predicted: Billing

[4] @105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or ...
    → Predicted: Billing

[5] @VirginTrains see attached error message. I've tried leaving a voicema...
    → Predicted: Account

[6] @105836 Have you tried from another device, Miriam ^MM...
    → Predicted: Technical

[7] @VirginTrains yep, I've tried laptop too several times over the past w...
    → Predicted: Technical

[8] @105836 It's working OK from here, Miriam. Does this link help https:/...
    → Predicted: Technical

[9] @VirginTrains I still haven't heard &amp; the number I'm directed to b...
    → Predicted: Technical

[10] @105836 That's wha